# DEEPX Tutorial 05 - DX-Compiler Workflow


In Tutorial 5, you will practice compiling classification, object detection, and segmentation models using the DX-Compiler. We will also review the guide for troubleshooting problems during compilation.

For more details, refer to the DX-Compiler user guide 👉 [Download](https://developer.deepx.ai/?files=MjY0NA==)

>This tutorial is based on dx-all-suite v2.1.0, released in December 2025.

## Compiling Image Classification Model (MobileNetV2)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [22]:
# Install pytorch & onnx
!pip install --quiet torch torchvision onnx onnxsim onnxscript netron portpicker tqdm seaborn

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

Export Pytorch based MobileNetV2 model to ONNX

In [ ]:
import torch, torchvision

# Load MobileNetV2 model
model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.eval()

# Batch size must be 1
dummy_input = torch.randn(1, 3, 224, 224)

onnx_path = "MobilenetV2.onnx"

torch.onnx.export(
    model,                      # PyTorch model object to export
    dummy_input,                # Dummy input used for tracing (tuple is possible)
    onnx_path,                  # Output ONNX file path
    export_params=True,         # If True, saves model parameter (weight) into the ONNX file
    input_names=["input_test"], # Name of the ONNX model input tensor
    output_names=["output"],    # Name of the ONNX model output tensor
    opset_version=13            # ONNX opset version (recommended: 11 ~ 21)
)
print("✅ Save ONNX:", onnx_path)

Clarify the **Input Name** and **Input Shape** using netron which see the AI model's topology:

In [ ]:
import netron
import os

# 1. Set port
port = 8081

# 2. Start Netron Server
netron.start('MobilenetV2.onnx', address=('localhost', port), browse=False)

# 3. Create Proxy URL
proxy_url = f"/proxy/{port}/" 

# 4. Display IFrame
from IPython.display import IFrame
IFrame(src=proxy_url, width='100%', height=800)

### 2. JSON configuration for Input/Pre-processing/Calibration

Generate a configuration file for Input/Pre-processing/Calibration of MobilenetV2.

This JSON configuration file incudes:
 - Input specifications
 - Calibration methods
 - Data preprocessing settings
 - Optional parameters for advanced compilation schemes

Model **Input Restrictions**:
 - The batch size must be fixed to 1
 - Only a single input is supported (Multi-input will be supported in 2026)
 - Input name must exactly match ONNX model definition

#### 2.1. Incorrect input name case

This MobilenetV2 model's input name is `input_test` as shown below:

![](assets/mobilenetv2-input-name.png)

Input name (`input_test` in this example) must exactly match the input name of JSON configuration.

However, you can see there is wrong input name `incorrect_input_name` in the following JSON configuration for inputs.

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"incorrect_input_name": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Let's run DX-Compiler with this wrong JSON configuration. You will meet ERROR:
> ConfigInputError: The input name in config incorrect_input_name is not same as model input input_test

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 2.2. Incorrect input shape
In the following JSON file, the input name has the correct one - `input_test`.

However, even if your input shape is 1x3x224x224 (BxCxHxW), calibration image shape has 224x224x3 (HxWxC). You must change the shape of calibraiton image to match with 1x3x224x224 (BxCxHxW) by using `transpose` and `expandDim`.

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Let's run DX-COM with this wrong JSON configuration. You will meet ERROR:

> ConfigInputError: Config shape [1, 3, 224, 224] does not match preprocessed data shape [1, 224, 224, 3]

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 2.3. Add `transpose` & `expanDim` to JSON

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

### 3. Compile with DX-Compiler and verify .dxnn
Compile with `dx_com` to generate `.dxnn`

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

In [ ]:
!run_model -m MobilenetV2.dxnn

## Compiling Object Detection Model (YOLOv9s)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [13]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

/home/max/Works/dx-tutorials/dx-all-suite/workspace/release/dx_com/dx_com_M1_v2.1.0


#### 1.1. Download `yolov9-s.pt` pytorch model to your local system:

In [ ]:
#!cd yolov9 && pip install -r requirements.txt
!pip install torch==2.5.1 torchvision==0.20.1
!pip install onnx onnxsim

In [14]:
!wget --no-check-certificate https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt

--2025-12-12 16:08:01--  https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt
Resolving github.com (github.com)... 20.200.245.247
Connecting to github.com (github.com)|20.200.245.247|:443... connected.
  Self-signed certificate encountered.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/759338070/5efccc50-06db-4a1f-bf78-db9fcce17d09?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-12T07%3A44%3A31Z&rscd=attachment%3B+filename%3Dyolov9-s.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-12T06%3A44%3A20Z&ske=2025-12-12T07%3A44%3A31Z&sks=b&skv=2018-11-09&sig=GXmOTdnUDBuzu7oXjo19cKfxeJefLnKJ0v8GWidgnvQ%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTUyNTA4MiwibmJmIjoxNzY1NTIzMjgyLCJwYXRoIjoicmV

#### 1.2. Export the downloaded torch model to ONNX:

Download yolov9 git repo to use `export.py` for yolov9.

In [15]:
!git clone https://github.com/WongKinYiu/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 781, done.
remote: Total 781 (delta 0), reused 0 (delta 0), pack-reused 781 (from 1)
Receiving objects: 100% (781/781), 3.27 MiB | 23.25 MiB/s, done.
Resolving deltas: 100% (330/330), done.


Export Pytorch based `yolov9-s` model to ONNX

In [23]:
!cd yolov9 && python3 export.py --weights ../yolov9-s.pt \
                                --img-size 640 640 \
                                --opset 12 \
                                --simplify \
                                --batch-size 1 \
                                --include onnx

/home/max/Works/dx-tutorials/dx-all-suite/workspace/release/dx_com/dx_com_M1_v2.1.0/yolov9/utils/general.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
export: data=data/coco.yaml, weights=['../yolov9-s.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.12.3 torch-2.9.1+cu128 CPU

Traceback (most recent call last):
  File "/home/max/Works/dx-tutorials/dx-all-suite/workspace/release/dx_com/dx_com_M1_v2.1.0/yolov9/export.py", line 686, in <module>
    main(opt)
  File "/home/max/Wo

Check if `yolov9-s.onnx` file is generated.

In [17]:
!ls | grep yolov9-s.onnx

### 2. JSON configuration for Input/Pre-processing/Calibration

In [18]:
%%writefile yolov9-s.json
{
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

Writing yolov9-s.json


### 3. Compile with DX-Compiler and verify .dxnn

In [19]:
!./dx_com/dx_com -m yolov9-s.onnx -c yolov9-s.json -o ./

[INFO] - Using optimization level 1. Compilation may take longer.
[INFO] - For faster compilation, consider using --opt_level 0 (may increase NPU latency).
[INFO] - Starting file validation...
[ERROR] - ========================================================================================================================
[ERROR] - Error in PhaseLabels.PREPARE: ONNXFileNotFoundError ONNX model file not found: yolov9-s.onnx
[ERROR] - ========================================================================================================================
Failed to compile yolov9-s.onnx.
Error Log:
	ONNXFileNotFoundError: ONNX model file not found: yolov9-s.onnx


Check if **yolov9-s.dxnn** file is generated as expected:

In [ ]:
!ls | grep yolov9-s.dxnn

In [ ]:
!run_model -m yolov9-s.dxnn

## Compiling ViT Model (WIP)

https://github.com/mlfoundations/open_clip


In [ ]:
!pip install -q open_clip_torch

In [ ]:
!pip install -q onnxruntime onnxsim onnx onnxscript

In [ ]:
import os
import sys
import torch
import numpy as np

import open_clip
from open_clip.zero_shot_classifier import build_zero_shot_classifier

def import_imagenet_metadata():
    try:
        from open_clip.zero_shot_metadata import (
            IMAGENET_CLASSNAMES,
            OPENAI_IMAGENET_TEMPLATES,
        )
        return IMAGENET_CLASSNAMES, OPENAI_IMAGENET_TEMPLATES
    except ImportError:
        print("\n[ERROR] Could not import IMAGENET_CLASSNAMES or OPENAI_IMAGENET_TEMPLATES.")
        print("This suggests a mismatch between your source code and the installed library in your conda environment.")
        sys.exit(1)

def main():
    device = 'cuda'#'cpu'
    # Model              | Pre-trained
    # ----------------------------------------
    # ViT-B-32-quickgelu | metaclip_fullcc
    # ViT-B-16-quickgelu | metaclip_fullcc
    # ViT-L-14-quickgelu | dfn2b
    # ViT-B-16           | dfn2b
    # ViT-L-14           | datacomp_xl_s13b_b90k
    # ViT-B-32-256       | datacomp_s34b_b86k
    # ViT-L-14-336       | openai
    model_name = 'ViT-L-14-quickgelu'
    pretrained = 'dfn2b'
    output_dir = 'out'
    output_path = os.path.join(output_dir, f"{model_name}-{pretrained}.npy")

    # Import classnames and templates
    classnames, templates = import_imagenet_metadata()

    # Load model and tokenizer
    print(f"Loading model '{model_name}' ({pretrained}) on '{device}'...")
    model, _, _ = open_clip.create_model_and_transforms(
        model_name, pretrained=pretrained, device=device
    )
    tokenizer = open_clip.get_tokenizer(model_name)
    model.eval()

    # Build zero-shot classifier weights
    print(f"Building zero-shot classifier weights on '{device}'...")
    with torch.no_grad():
        zeroshot_weights = build_zero_shot_classifier(
            model,
            tokenizer=tokenizer,
            classnames=classnames,
            templates=templates,
            device=device,
        )

    print(f"Zero-shot weights shape: {zeroshot_weights.shape}, dtype: {zeroshot_weights.dtype}")

    # Save as numpy
    os.makedirs(output_dir, exist_ok=True)
    np.save(output_path, zeroshot_weights.cpu().numpy())
    print(f"Zero-shot weights saved to: {output_path}")
    print("Done!")

if __name__ == "__main__":
    main()

In [ ]:
!echo "hi"

In [ ]:
!ls -lah out

In [ ]:
import onnx
import onnxsim
from PIL import Image

def parse_preprocess(preprocess):
    dx_preprocess = []

    for trf in preprocess.transforms:
        trf_name = trf.__class__.__name__

        if trf_name == "Resize":
            size = trf.size
            if isinstance(size, int):
                dx_preprocess.append({"resize": {"width": size, "height": size}})
            else:
                dx_preprocess.append({"resize": {"width": size[0], "height": size[1]}})
        elif trf_name == "ToTensor":
            dx_preprocess.append({"div": {"x": 255.0}})
        elif trf_name == "Normalize":
            mean = trf.mean
            std = trf.std
            dx_preprocess.append(
                {"normalize": {"mean": mean, "std": std}},
            )
        elif trf_name == "function":
            trf_name = trf.__name__
            if trf_name == "_convert_to_rgb":
                dx_preprocess.append({"convertColor": {"form": "BGR2RGB"}})
            else:
                raise NotImplementedError(trf_name)
        elif trf_name == "function":
            trf_name = trf.__name__
            if trf_name == "_convert_to_rgb":
                dx_preprocess.append({"convertColor": {"form": "BGR2RGB"}})
            else:
                raise NotImplementedError(trf_name)

        elif trf_name == "CenterCrop":
            size = trf.size

            if isinstance(size, int):
                dx_preprocess.append({"centercrop": {"width": size, "height": size}})
            else:
                dx_preprocess.append({"centercrop": {"width": size[0], "height": size[1]}})

        else:
            raise NotImplementedError(trf_name)
    return dx_preprocess

def get_config(img: torch.Tensor, preprocess):
    template = {
        "inputs": {"input": [1, 3, 224, 224]},
        "default_loader": {
            "dataset_path": "/mnt/datasets/calibration_dataset/",
            "file_extensions": ["jpeg", "jpg", "png", "JPEG"],
        },
        "calibration_num": 100,
        "calibration_method": "ema",
        "train_batchsize": 32,
        "num_samples": 100,
    }
    template["inputs"]["input"] = img.shape
    _preprocess = parse_preprocess(preprocess)
    _preprocess.append({"transpose": {"axis": [2, 0, 1]}})
    _preprocess.append({"expandDim": {"axis": 0}})

    template["default_loader"]["preprocessings"] = _preprocess
    return template

def main():
    output_dir = 'out'
    model_name = 'ViT-L-14-quickgelu'
    pretrained = 'dfn2b'
    output_path = os.path.join(output_dir, f"{model_name}-{pretrained}.onnx")
    config_path = os.path.join(output_dir, f"{model_name}-{pretrained}.json")
    transform_path = os.path.join(output_dir, f"{model_name}-{pretrained}-transform.txt")

    # Load model and tokenizer
    print(f"Loading model '{model_name}' ({pretrained}) ...")
    model, _, transform = open_clip.create_model_and_transforms(
        model_name, pretrained=pretrained
    )
    model.eval()

    image = transform(Image.open("assets/CLIP.png")).unsqueeze(0)

    os.makedirs(output_dir, exist_ok=True)
    torch.onnx.export(model.visual, image, output_path, opset_version=17)
    with open(transform_path, "w") as f:
        f.write(str(transform))
    print(f"ONNX model saved to: {output_path}")
    config = get_config(image, transform)
    with open(config_path, "w") as f:
        import json
        json.dump(config, f)
    print(f"Config file saved to: {config_path}")

    print("Simplifying ONNX model ...")
    onnx_model = onnxsim.simplify(output_path)[0]
    onnx.save(onnx_model, output_path)
    print(f"Simplified ONNX model saved to: {output_path}")


if __name__ == "__main__":
    main()

In [ ]:
!ls -lah out

In [ ]:
./dx_com/dx_com -m sample/ViT-L-14-quickgelu-dfn2b.onnx -c sample/ViT-L-14-quickgelu-dfn2b.json  -o out
Compiling Model : 100%|██████████████████████████████████████████████████████████████| 1.0/1.0 [01:17<00:00, 43.33s/model ]Failed to compile sample/ViT-L-14-quickgelu-dfn2b.onnx.
Error Log:
	TypeError: Sequential.__init__() takes 2 positional arguments but 3 were given